In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os
from matplotlib.pyplot import hist
# import more functions or modules if you need them !!

In [2]:
# set random seed for numpy
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

In [6]:
oil_df = pd.read_pickle('../Data/data/comprehensive_new_full.pkl')

In [7]:
oil_df

,year,numcode,oilreserves_full,oilreserves,oilreserves_public,newdiscovery_aspo,aspo,wildcat,endowment,pop_maddison,...,valoilres_binarize,valoilres_public_diff,valoilres_public_binarize,oilpop_diff,oilpop_binarize,valoilres_impute_diff,valoilres_impute_binarize,oilpop_impute_diff,oilpop_impute_binarize,milexp_pergdpSIPRI_diff
0,1929.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1931.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1932.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1933.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17915,2004.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,10962.026367,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17916,2005.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,11115.380859,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17917,2006.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,11288.252930,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17918,2007.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,11477.447266,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [8]:
oil_df.columns

Index(['year', 'numcode', 'oilreserves_full', 'oilreserves',
       'oilreserves_public', 'newdiscovery_aspo', 'aspo', 'wildcat',
       'endowment', 'pop_maddison', 'ecgrowth', 'efrac', 'lfrac', 'rfrac',
       'incidence2COW', 'onset2COWCS', 'incidenceU', 'onsetUCS', 'Fearon_war',
       'onset_FearonCS', 'Sambanis_war', 'onset_SambanisCS', 'coup',
       'any_coup', 'periregular', 'no_transition', 'milexp_pergdpSIPRI',
       'mountain', 'religion_fractionalization', 'ethnic_fractionalization',
       'language_fractionalization', 'ETHPOL_reynal', 'ETHFRAC_reynal',
       'RELPOL_reynal', 'RELFRAC_reynal', 'leg_british', 'rule_law96',
       'land_area', 'crude1990P', 'smallregion', 'largeregion', 'code3',
       'out_regdisaster', 'oilpop', 'zero_res', 'logoilres', 'oilpop_public',
       'logoilres_public', 'logoilres_full', 'valoilres', 'valoilres_public',
       'logvaloilres', 'logvaloilres_public', 'logvaloilres_full',
       'oilpop_impute', 'logoilres_impute', 'valoilres_imp

In [9]:
# ATT and ATE AIPTW
def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    Double ML estimator for the ATT
    This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
    Return: aiptw of ATE and its standard error
    """
    
    # number of observations
    n = Y.shape[0]
    
    # estimate marginal probability of treatment
    if prob_t is None:
        prob_t = A.mean() 
    
    # att aiptw
    tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
    # influence curve and standard error of aiptw
    phi = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
    std_hat = np.std(phi) / np.sqrt(n)

    return tau_hat, std_hat

def ate_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    Double ML estimator for the ATE
    Return: aiptw of ATE and its standard error
    """
    # number of observations
    n = Y.shape[0]
    
    # ate aiptw
    tau_hat = (Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g)).mean()
  
    # influence curve and standard error of aiptw
    phi = Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g) - tau_hat   
    std_hat = np.std(phi) / np.sqrt(n)

    return tau_hat, std_hat

In [10]:
# Conditional outcome models (Q models)
def make_linear_Q_model():
    ''' A function that returns a linear q model for later use in k-folding'''
    return LinearRegression()

def make_Q_model(output_type:str):
    ''' A function that returns a general ML q model for later use in k-folding'''
    if output_type == 'binary':
        return RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
    return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
# One example: RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)

In [11]:
# Propensity score models (g models)
def make_g_model():
    ''' A function that returns a g model for computing propensity scores'''
    return RandomForestClassifier(n_estimators=100, max_depth=5)
# One example: RandomForestClassifier(n_estimators=100, max_depth=5)

In [68]:
# Functions for K-fold cross-fitting
def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    '''
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    '''

    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
        X_train = X.loc[train_index]
        A_train = A.loc[train_index]
        g = make_model()
        g.fit(X_train, A_train)

        # get predictions for split
        predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]
    
    # sanity check that overlap holds
    assert np.isnan(predictions).sum() == 0
    return predictions

def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    '''
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"
    '''

    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
        X_train = X_w_treatment.loc[train_index]
        y_train = y.loc[train_index]
        q = make_model(output_type)
        q.fit(X_train, y_train)

        if output_type =='binary':
            predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
            predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
        elif output_type == 'continuous':
            predictions0[test_index] = q.predict(X0.loc[test_index])
            predictions1[test_index] = q.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [69]:
model_vars = ['onset2COWCS',
              'valoilres_binarize',
              'ecgrowth',
              'pop_maddison_diff',
              'popdens_diff',
              'democracy_diff',
              'logmountain',
              'ethnic_fractionalization',
              'religion_fractionalization',
              'language_fractionalization',
              'leg_british',
              'numcode',
              'year']

In [70]:
outcome = 'onset2COWCS'
treatment = 'valoilres_binarize'
confounders = [x for x in model_vars if x not in (outcome + treatment)]

In [84]:
def fit_and_run_model(df, outcome:str, treatment:str, confounders:list, make_g_model,
                      make_Q_model, n_splits=5, output_type='binary', ate=True):
    '''
    Function that creates a g, q, and aiptw model based on the 
    given inputs
    
    Inputs: df (pandas df) - the dataframe the variables are contained in
            outcome (str) - the outcome variable
            treatment (str) - the treatment variable
            confounders (lst) - a list of the confounding variables
            make_g_model - the make_g_model function
            make_Q_model - the make_Q_model function
            n_splits (int) - number of splits for the model
            output_type (str) - the desired output type, either binary or continous
            ate (bool) - whether to use ate or alternative att
    
    Returns: tau_hat - the tau hat estimator for the average treatment effect
             std of tau_hat - the standard deviation for the tau_hat estimator
    '''
    df = df.replace({outcome: .00001}, 0)
    df = df[[outcome] + confounders + [treatment]]
    df = df.dropna().reset_index()
    print('Running models for treatment {} and outcome {} on {} samples'.format(treatment, outcome, len(df)))
    outcome = df[outcome]
    confounders = df[confounders]
    treatment = df[treatment]
    
    g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=n_splits)
    
    if min(g) < .01:
        print('\nWARNING:\n Some propensity scores are very small,\n which could '
              'lead to an inflated AIPTW.\n Minimum score = ', min(g))
    if max(g) > .99:
        print('\nWARNING:\n Some propensity scores are very large,\n which could '
              'lead to an inflated AIPTW.\n Maximum score = ', max(g))
    print('G Model has been fit')

    Q0_ml, Q1_ml = outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, \
                                                  n_splits=n_splits, output_type=output_type)
    
    print('Q model has been fit')
    data_and_nuisance_estimates_ml = pd.DataFrame({'g': g, 'Q0': Q0_ml, 'Q1': Q1_ml, 'A': treatment, 'Y': outcome})
    
    # ate aiptw
    if ate:
        tau_hat, std_hat = ate_aiptw(**data_and_nuisance_estimates_ml)
    else: 
        tau_hat, std_hat = att_aiptw(**data_and_nuisance_estimates_ml)
    print('AIPTW model has been fit. Returning \u03C4 hat and its standard deviation')
    print('\u03C4 hat = {} and std = {}\n'.format(round(tau_hat, 5), round(std_hat, 5)))
    return tau_hat, std_hat

In [72]:
treatment_lst = ['valoilres_binarize', # value of oil reserves
               'valoilres_public_binarize', # value of oil reserves from public data
               'oilpop_binarize', # oil reserves per capita in million barrels per 1000 persons
               'valoilres_impute_binarize', # value of oilpop_impute (multiply by crude oil price)
               'oilpop_impute_binarize']

outcome_lst = ['onset2COWCS',
               'onsetUCS',
               'coup',
               'periregular',
               'milexp_pergdpSIPRI_diff']

In [73]:
undemocratic_countries_df = oil_df[oil_df['democracy'] <= oil_df['democracy'].median()]
democratic_countries_df = oil_df[oil_df['democracy'] > oil_df['democracy'].median()]

In [77]:
df_dict = {'model': [], 'treatment': [], 'outcome': [], 'tau_hat': [], 'tau_std': []}
for treat in treatment_lst:
    for out in outcome_lst:
        if len(oil_df[out].value_counts()) == 2:
            output = 'binary'
        else:
            output = 'continuous'
        tau_hat, tau_std = fit_and_run_model(oil_df, out, treat, confounders, make_g_model, make_Q_model, output_type=output)
        df_dict['model'].append('ate')
        df_dict['treatment'].append(treat)
        df_dict['outcome'].append(out)
        df_dict['tau_hat'].append(tau_hat)
        df_dict['tau_std'].append(tau_std)

ate_results_df = pd.DataFrame(data=df_dict)

Running models for treatment valoilres_binarize and outcome onset2COWCS on 5174 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.0043 and std = 0.00451

Running models for treatment valoilres_binarize and outcome onsetUCS on 4754 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.01022 and std = 0.0077

Running models for treatment valoilres_binarize and outcome coup on 5364 samples

 Some propensity scores are very small,
 which could lead to an inflated AIPTW.
 Minimum score =  0.009121221447996871
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = -0.00336 and std = 0.01154

Running models for treatment valoilres_binarize and outcome periregular on 3312 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard 

In [78]:
df_dict = {'model': [], 'treatment': [], 'outcome': [], 'tau_hat': [], 'tau_std': []}
for treat in treatment_lst:
    for out in outcome_lst:
        if len(oil_df[out].value_counts()) == 2:
            output = 'binary'
        else:
            output = 'continuous'
        tau_hat, tau_std = fit_and_run_model(oil_df, out, treat, confounders, make_g_model, make_Q_model, output_type=output, ate=False)
        df_dict['model'].append('att')
        df_dict['treatment'].append(treat)
        df_dict['outcome'].append(out)
        df_dict['tau_hat'].append(tau_hat)
        df_dict['tau_std'].append(tau_std)

att_results_df = pd.DataFrame(data=df_dict)

Running models for treatment valoilres_binarize and outcome onset2COWCS on 5174 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00274 and std = 0.00498

Running models for treatment valoilres_binarize and outcome onsetUCS on 4754 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00645 and std = 0.00867

Running models for treatment valoilres_binarize and outcome coup on 5364 samples

 Some propensity scores are very small,
 which could lead to an inflated AIPTW.
 Minimum score =  0.007276610576749231
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = -0.01214 and std = 0.01315

Running models for treatment valoilres_binarize and outcome periregular on 3312 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standar

In [81]:
results_df = pd.concat([ate_results_df, att_results_df])
results_df

,model,treatment,outcome,tau_hat,tau_std
0,ate,valoilres_binarize,onset2COWCS,0.004296,0.004511
1,ate,valoilres_binarize,onsetUCS,0.010222,0.007701
2,ate,valoilres_binarize,coup,-0.003363,0.011545
3,ate,valoilres_binarize,periregular,-0.012247,0.004606
4,ate,valoilres_binarize,milexp_pergdpSIPRI_diff,0.449256,0.382961
5,ate,valoilres_public_binarize,onset2COWCS,0.002779,0.004407
6,ate,valoilres_public_binarize,onsetUCS,0.017846,0.006766
7,ate,valoilres_public_binarize,coup,0.023525,0.011499
8,ate,valoilres_public_binarize,periregular,-0.007588,0.006317
9,ate,valoilres_public_binarize,milexp_pergdpSIPRI_diff,0.295875,0.224785


In [96]:
df_dict = {'model': [], 'democracy': [], 'treatment': [], 'outcome': [], 'tau_hat': [], 'tau_std': []}
for treat in treatment_lst:
    for ate_bool in [True, False]:
        for democracy in [True, False]:
            if democracy:
                df = oil_df[oil_df['democracy'] > .005]
            else:
                df = oil_df[oil_df['democracy'] < .005]
            tau_hat, tau_std = fit_and_run_model(df, 'milexp_pergdpSIPRI_diff', treat, confounders, make_g_model, make_Q_model, output_type='continuous', ate=False)
            if ate_bool:
                df_dict['model'].append('ate')
            else:
                df_dict['model'].append('att')
            df_dict['treatment'].append(treat)
            df_dict['outcome'].append(out)
            df_dict['tau_hat'].append(tau_hat)
            df_dict['tau_std'].append(tau_std)
            df_dict['democracy'].append(democracy)

defense_burden_df = pd.DataFrame(data=df_dict)

Running models for treatment valoilres_binarize and outcome milexp_pergdpSIPRI_diff on 809 samples

 Some propensity scores are very small,
 which could lead to an inflated AIPTW.
 Minimum score =  0.007295462792614008
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00136 and std = 0.02996

Running models for treatment valoilres_binarize and outcome milexp_pergdpSIPRI_diff on 459 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 1.12665 and std = 1.15774

Running models for treatment valoilres_binarize and outcome milexp_pergdpSIPRI_diff on 809 samples

 Some propensity scores are very small,
 which could lead to an inflated AIPTW.
 Minimum score =  0.005278227822055531
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00518 and std = 0.02987

Running models for treatm

In [97]:
defense_burden_df

,model,democracy,treatment,outcome,tau_hat,tau_std
0,ate,True,valoilres_binarize,milexp_pergdpSIPRI_diff,0.001358,0.029957
1,ate,False,valoilres_binarize,milexp_pergdpSIPRI_diff,1.126646,1.157744
2,att,True,valoilres_binarize,milexp_pergdpSIPRI_diff,0.005178,0.029865
3,att,False,valoilres_binarize,milexp_pergdpSIPRI_diff,1.201806,1.172956
4,ate,True,valoilres_public_binarize,milexp_pergdpSIPRI_diff,-0.002583,0.028169
5,ate,False,valoilres_public_binarize,milexp_pergdpSIPRI_diff,1.206849,0.937151
6,att,True,valoilres_public_binarize,milexp_pergdpSIPRI_diff,-0.003343,0.028158
7,att,False,valoilres_public_binarize,milexp_pergdpSIPRI_diff,1.208276,0.940733
8,ate,True,oilpop_binarize,milexp_pergdpSIPRI_diff,0.109423,0.099705
9,ate,False,oilpop_binarize,milexp_pergdpSIPRI_diff,6.531253,6.422997
